In [6]:
#packages
import json
import tqdm
import os
import numpy as np
import sys
import glob
import pandas as pd
import datetime as dt
import re
import math
from collections import defaultdict
import operator
import random

#plot
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
PACKAGE_PARENT = '../'
SCRIPT_DIR = os.path.dirname(os.path.realpath(os.path.join(os.getcwd(), os.path.expanduser('__file__'))))
sys.path.append(os.path.normpath(os.path.join(SCRIPT_DIR, PACKAGE_PARENT)))
from UTILS import preprocessing_Origins, general_cleaning, verification_based_on_initial_record,\
ZoneVariable, HenDailyVariable_Origins, boxplot_distribution_entropy, heatmap_duration_perzone_perhen, is_day
import config_origins as config

C:\Users\camil\anaconda3\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


2


In [6]:
#initialise parameter
path_initial_data = config.path_initial_data
path_extracted_data = config.path_extracted_data
path_extracted_data_daily_check = config.path_extracted_data_daily_check
id_run = config.id_run
dico_rc_sys = config.dico_rc_sys
path_FocalBird = config.path_FocalBird

focal_name = config.focal_name
path_Days = config.path_Days

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Find disfunctioning tags

In [7]:
df_wt = pd.read_csv(os.path.join(path_extracted_data, id_run+'_df_alldeviceinfo.csv'), parse_dates=['date'], dayfirst=True, sep=';') 
print(df_wt.shape)
df_wt.head(3)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(46566, 12)


,Unnamed: 0,sender,date,LFCounter,LFCounter_nbr_equal0,LFCounter_atleastone0,gap,bigest_gap,nbr_obs,expected_nbr_obs,is_nbr_obs_correst,2keep
0,0,tag_10,2020-10-13,"[232.0, 232.0, 232.0, 232.0, 232.0, 232.0, 232...",0,False,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",120.0,600,[71],True,True
1,1,tag_10,2020-10-14,"[266.0, 266.0, 266.0, 266.0, 266.0, 266.0, 266...",0,False,"[0.0, 1.0, 1.0, 1.0, 1.0, 2.0, 2.0, 2.0, 2.0, ...",122.0,600,[71],True,True
2,2,tag_10,2020-10-15,"[273.0, 273.0, 273.0, 273.0, 273.0, 273.0, 273...",0,False,"[0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 2.0, 2.0, ...",122.0,600,[71],True,True


In [15]:
df_tag = df_wt.groupby(['sender'])[['date','bigest_gap','LFCounter_nbr_equal0','nbr_obs','is_nbr_obs_correst','2keep']].agg(lambda x: list(x)).reset_index()
print(df_tag.shape)
df_tag.head(3)

(176, 7)


C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,sender,date,bigest_gap,LFCounter_nbr_equal0,nbr_obs,is_nbr_obs_correst,2keep
0,tag_10,"[2020-10-13 00:00:00, 2020-10-14 00:00:00, 202...","[120.0, 122.0, 122.0, 121.0, 122.0, 122.0, 122...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[600, 600, 600, 660, 660, 660, 660, 659, 660, ...","[True, True, True, True, True, True, True, Tru...","[True, True, True, True, True, True, True, Tru..."
1,tag_100,"[2020-10-13 00:00:00, 2020-10-14 00:00:00, 202...","[120.0, 120.0, 123.0, 122.0, 123.0, 123.0, 123...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[600, 600, 600, 660, 660, 660, 660, 660, 660, ...","[True, True, True, True, True, True, True, Tru...","[True, True, True, True, True, True, True, Tru..."
2,tag_102,"[2020-10-13 00:00:00, 2020-10-14 00:00:00, 202...","[120.0, 120.0, 122.0, 123.0, 123.0, 122.0, 123...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[600, 600, 600, 660, 660, 660, 660, 660, 660, ...","[True, True, True, True, True, True, True, Tru...","[True, True, True, True, True, True, True, Tru..."


In [16]:
df_tag['perc_2keep'] = df_tag['2keep'].map(lambda x: sum(x)/len(x)*100)
df_tag['perc_LFCountermorethan5_0'] = df_tag['LFCounter_nbr_equal0'].map(lambda x: sum([i>=config.lf_counter for i in x])/len(x)*100)
df_tag['perc_badgap'] = df_tag['bigest_gap'].map(lambda x: sum([i>(12*60) for i in x])/len(x)*100)
df_tag.head(3)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,sender,date,bigest_gap,LFCounter_nbr_equal0,nbr_obs,is_nbr_obs_correst,2keep,perc_2keep,perc_LFCountermorethan5_0,perc_badgap
0,tag_10,"[2020-10-13 00:00:00, 2020-10-14 00:00:00, 202...","[120.0, 122.0, 122.0, 121.0, 122.0, 122.0, 122...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[600, 600, 600, 660, 660, 660, 660, 659, 660, ...","[True, True, True, True, True, True, True, Tru...","[True, True, True, True, True, True, True, Tru...",60.451977,38.418079,2.824859
1,tag_100,"[2020-10-13 00:00:00, 2020-10-14 00:00:00, 202...","[120.0, 120.0, 123.0, 122.0, 123.0, 123.0, 123...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[600, 600, 600, 660, 660, 660, 660, 660, 660, ...","[True, True, True, True, True, True, True, Tru...","[True, True, True, True, True, True, True, Tru...",94.468085,1.702128,4.255319
2,tag_102,"[2020-10-13 00:00:00, 2020-10-14 00:00:00, 202...","[120.0, 120.0, 122.0, 123.0, 123.0, 122.0, 123...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[600, 600, 600, 660, 660, 660, 660, 660, 660, ...","[True, True, True, True, True, True, True, Tru...","[True, True, True, True, True, True, True, Tru...",83.623693,13.937282,14.982578


In [17]:
df_tag.sort_values(['perc_2keep']).head(30)
#issue:tag 88, 51, 71, 109
#tag 10 stoped working after an HA
#tag 

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,sender,date,bigest_gap,LFCounter_nbr_equal0,nbr_obs,is_nbr_obs_correst,2keep,perc_2keep,perc_LFCountermorethan5_0,perc_badgap
165,tag_88,"[2020-10-13 00:00:00, 2020-10-14 00:00:00, 202...","[120.0, 120.0, 122.0, 121.0, 121.0, 121.0, 121...","[600, 600, 600, 660, 660, 660, 660, 659, 660, ...","[600, 600, 600, 660, 660, 660, 660, 659, 660, ...","[True, True, True, True, True, True, True, Tru...","[False, False, False, False, False, False, Fal...",0.000000,100.000000,94.350282
130,tag_51,"[2020-10-13 00:00:00, 2020-10-14 00:00:00, 202...","[120.0, 123.0, 122.0, 122.0, 122.0, 123.0, 123...","[285, 182, 21, 91, 10, 101, 10, 86, 356, 132, ...","[600, 600, 600, 660, 660, 660, 660, 659, 660, ...","[True, True, True, True, True, True, True, Tru...","[False, False, False, False, False, False, Fal...",0.000000,100.000000,94.350282
151,tag_71,"[2021-04-16 00:00:00, 2021-04-17 00:00:00, 202...","[118.0, 113.0, 115.0, 117.0, 115.0, 114.0, 116...","[0, 0, 0, 4, 3, 7, 2, 3, 3, 7, 0, 0, 0, 0, 0, ...","[90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 9...","[True, True, True, True, True, True, True, Tru...","[True, True, True, True, True, False, True, Tr...",7.843137,1.960784,90.196078
8,tag_109,"[2020-10-13 00:00:00, 2020-10-14 00:00:00, 202...","[120.0, 120.0, 122.0, 123.0, 122.0, 122.0, 122...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[600, 600, 600, 660, 660, 660, 660, 659, 660, ...","[True, True, True, True, True, True, True, Tru...","[True, True, True, True, True, True, True, Tru...",23.163842,76.836158,76.271186
150,tag_70,"[2020-10-13 00:00:00, 2020-10-14 00:00:00, 202...","[120.0, 120.0, 120.0, 121.0, 121.0, 121.0, 120...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[600, 600, 600, 660, 660, 660, 660, 660, 660, ...","[True, True, True, True, True, True, True, Tru...","[True, True, True, True, True, True, True, Tru...",23.728814,76.271186,53.672316
160,tag_80,"[2020-10-13 00:00:00, 2020-10-14 00:00:00, 202...","[120.0, 120.0, 124.0, 124.0, 124.0, 124.0, 124...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[600, 600, 600, 660, 660, 660, 660, 660, 660, ...","[True, True, True, True, True, True, True, Tru...","[True, True, True, True, True, True, True, Tru...",25.423729,63.841808,71.186441
152,tag_72,"[2020-10-13 00:00:00, 2020-10-14 00:00:00, 202...","[120.0, 120.0, 121.0, 120.0, 120.0, 120.0, 121...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[600, 600, 600, 660, 660, 660, 660, 660, 660, ...","[True, True, True, True, True, True, True, Tru...","[True, True, True, True, True, True, True, Tru...",45.762712,33.333333,53.672316
0,tag_10,"[2020-10-13 00:00:00, 2020-10-14 00:00:00, 202...","[120.0, 122.0, 122.0, 121.0, 122.0, 122.0, 122...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[600, 600, 600, 660, 660, 660, 660, 659, 660, ...","[True, True, True, True, True, True, True, Tru...","[True, True, True, True, True, True, True, Tru...",60.451977,38.418079,2.824859
161,tag_82,"[2020-10-13 00:00:00, 2020-10-14 00:00:00, 202...","[120.0, 120.0, 123.0, 123.0, 123.0, 123.0, 123...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[600, 600, 600, 660, 660, 660, 660, 660, 660, ...","[True, True, True, True, True, True, True, Tru...","[True, True, True, True, True, True, True, Tru...",68.361582,28.248588,4.519774
24,tag_130,"[2020-10-15 00:00:00, 2020-10-16 00:00:00, 202...","[122.0, 123.0, 123.0, 122.0, 122.0, 216.0, 123...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[600, 660, 660, 660, 660, 658, 660, 720, 720, ...","[True, True, True, True, True, True, True, Tru...","[True, True, True, True, True, True, True, Tru...",76.491228,3.859649,21.052632


In [18]:
df_tag.sort_values(['perc_2keep']).tail(5)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,sender,date,bigest_gap,LFCounter_nbr_equal0,nbr_obs,is_nbr_obs_correst,2keep,perc_2keep,perc_LFCountermorethan5_0,perc_badgap
142,tag_63,"[2020-10-13 00:00:00, 2020-10-14 00:00:00, 202...","[120.0, 121.0, 121.0, 121.0, 121.0, 121.0, 121...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[600, 600, 600, 660, 660, 660, 660, 659, 660, ...","[True, True, True, True, True, True, True, Tru...","[True, True, True, True, True, True, True, Tru...",97.212544,1.393728,2.090592
16,tag_121,"[2020-10-23 00:00:00, 2020-10-24 00:00:00, 202...","[120.0, 121.0, 119.0, 121.0, 119.0, 121.0, 119...","[720, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[720, 72, 72, 72, 72, 72, 72, 78, 78, 78, 78, ...","[True, True, True, True, True, True, True, Tru...","[False, True, True, True, True, True, True, Tr...",97.472924,1.444043,1.444043
80,tag_186,"[2020-10-13 00:00:00, 2020-10-14 00:00:00, 202...","[120.0, 121.0, 121.0, 121.0, 121.0, 121.0, 121...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[600, 600, 600, 660, 660, 660, 660, 659, 660, ...","[True, True, True, True, True, True, True, Tru...","[True, True, True, True, True, True, True, Tru...",97.560976,1.393728,1.742160
143,tag_64,"[2020-10-13 00:00:00, 2020-10-14 00:00:00, 202...","[120.0, 123.0, 123.0, 123.0, 124.0, 123.0, 123...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[600, 600, 600, 660, 660, 660, 660, 659, 660, ...","[True, True, True, True, True, True, True, Tru...","[True, True, True, True, True, True, True, Tru...",97.560976,1.393728,1.742160
125,tag_47,"[2020-10-13 00:00:00, 2020-10-14 00:00:00, 202...","[120.0, 121.0, 121.0, 121.0, 121.0, 121.0, 121...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[600, 600, 600, 660, 660, 660, 660, 659, 660, ...","[True, True, True, True, True, True, True, Tru...","[True, True, True, True, True, True, True, Tru...",97.560976,1.393728,1.393728


# Select focal hens randomly

In [2]:
(8*225) / 4

450.0

In [3]:
450+141

590

In [119]:
#we have 100 sires (each with ~40 daughters), from which we have too select randomly our 168 focal birds (21/pen)
#to do so we will for each pen select randomly 21 sireID

#fix random seed for reproducibility
random.seed(123)

#with repeatition, as we have 50 daughters per 100 sire, and we only need 168 focal birds in total
#some sires had almost no daughters <8 so we remove them (among those 4 whad all their daughters taken for hentrack): ending up into a random selection out of the 100 sire
li_focalhens_initial = random.choices(list([i for i in range(1,101) if i not in [96,81,45,41,31,14,18,66,88]]), k=168)
#print(li_focalhens_initial)

#randomly pick more than one element from the list without repeating elements.
li_focalhens_remaining = li_focalhens_initial
dico_ = {'pen3':list(np.repeat('Rpen11',6))+list(np.repeat('Rpen12',5))+list(np.repeat('Rpen13',5))+list(np.repeat('Rpen14',5)),
         'pen4':list(np.repeat('Rpen11',5))+list(np.repeat('Rpen12',6))+list(np.repeat('Rpen13',5))+list(np.repeat('Rpen14',5)),
         'pen5':list(np.repeat('Rpen11',5))+list(np.repeat('Rpen12',5))+list(np.repeat('Rpen13',6))+list(np.repeat('Rpen14',5)),
         'pen8':list(np.repeat('Rpen11',5))+list(np.repeat('Rpen12',5))+list(np.repeat('Rpen13',5))+list(np.repeat('Rpen14',6)),
         'pen9':list(np.repeat('Rpen11',6))+list(np.repeat('Rpen12',5))+list(np.repeat('Rpen13',5))+list(np.repeat('Rpen14',5)),
         'pen10':list(np.repeat('Rpen11',5))+list(np.repeat('Rpen12',6))+list(np.repeat('Rpen13',5))+list(np.repeat('Rpen14',5)),
         'pen11':list(np.repeat('Rpen11',5))+list(np.repeat('Rpen12',5))+list(np.repeat('Rpen13',6))+list(np.repeat('Rpen14',5)),
         'pen12':list(np.repeat('Rpen11',5))+list(np.repeat('Rpen12',5))+list(np.repeat('Rpen13',5))+list(np.repeat('Rpen14',6))}
li_df = []
for k,penID in enumerate(['pen3','pen4','pen5','pen8','pen9','pen10','pen11','pen12']):
    li_selected = random.sample(li_focalhens_initial, 21)
    li_focalhens_remaining = [i for i in li_focalhens_remaining if i not in li_selected]
    li_rearingPenID = dico_[penID]
    #print(len(li_focalhens_remaining))
    for p,henID in enumerate(li_selected):
        li_df.append({'SireID':henID, 'RPenID':li_rearingPenID[p],'PenID':penID,'legringID':p+1})
df_focalbirds = pd.DataFrame(li_df)
dico_RP_tr = {'Rpen11':'Queue', 'Rpen12':'Queue', 'Rpen13':'NoQueue', 'Rpen14':'NoQueue'}
df_focalbirds['Rtreatment'] = df_focalbirds['RPenID'].map(lambda x: dico_RP_tr[x])
dico_LP_leg = {'pen3':'yellow','pen4':'brightgreen','pen5':'black','pen8':'darkblue',
               'pen9':'violette','pen10':'red','pen11':'lightblue','pen12':'pink'}
df_focalbirds['LPLegring'] = df_focalbirds['PenID'].map(lambda x: dico_LP_leg[x])

df_focalbirds.to_csv(os.path.join(r'G:\VPHI\Welfare\2- Research Projects\Camille Montalcini\Origins.GS\FOCAL BIRDS','Focalbirds.csv'), index=False, sep=';')
df_focalbirds.to_csv(os.path.join(r'G:\VPHI\Welfare\2- Research Projects\Camille Montalcini\Origins.GS\FOCAL BIRDS','Focalbirds_comma.csv'), index=False, sep=',')

print(df_focalbirds.shape)
df_focalbirds.head(3)

(168, 6)


,SireID,RPenID,PenID,legringID,Rtreatment,LPLegring
0,53,Rpen11,pen3,1,Queue,yellow
1,39,Rpen11,pen3,2,Queue,yellow
2,34,Rpen11,pen3,3,Queue,yellow


In [120]:
df_focalbirds['PenID'].value_counts()

pen10    21
pen5     21
pen8     21
pen4     21
pen9     21
pen3     21
pen12    21
pen11    21
Name: PenID, dtype: int64

In [121]:
df_focalbirds['RPenID'].value_counts()

Rpen14    42
Rpen13    42
Rpen11    42
Rpen12    42
Name: RPenID, dtype: int64

In [122]:
df_check = df_focalbirds.groupby(['PenID','Rtreatment'])['SireID'].agg(lambda x: list(x)).reset_index()
df_check['nbr_hens'] = df_check['SireID'].map(lambda x: len(x))
df_check

,PenID,Rtreatment,SireID,nbr_hens
0,pen10,NoQueue,"[63, 78, 58, 6, 100, 87, 51, 97, 99, 7]",10
1,pen10,Queue,"[67, 82, 34, 91, 67, 30, 19, 46, 6, 77, 47]",11
2,pen11,NoQueue,"[74, 44, 52, 39, 53, 19, 34, 9, 16, 63, 17]",11
3,pen11,Queue,"[35, 75, 98, 53, 84, 8, 90, 24, 78, 8]",10
4,pen12,NoQueue,"[2, 8, 50, 3, 46, 84, 63, 93, 36, 34, 87]",11
5,pen12,Queue,"[32, 64, 54, 42, 89, 78, 26, 52, 34, 34]",10
6,pen3,NoQueue,"[100, 35, 8, 8, 21, 21, 79, 1, 1, 65]",10
7,pen3,Queue,"[53, 39, 34, 34, 23, 27, 63, 10, 47, 13, 21]",11
8,pen4,NoQueue,"[43, 40, 13, 72, 50, 55, 7, 7, 19, 79]",10
9,pen4,Queue,"[1, 72, 13, 44, 39, 53, 10, 19, 38, 99, 8]",11


In [123]:
df_check = df_focalbirds.groupby(['PenID','RPenID'])['SireID','Rtreatment'].agg(lambda x: list(x)).reset_index()
df_check['nbr_hens'] = df_check['SireID'].map(lambda x: len(x))
df_check

<ipython-input-123-8f8d8430fcda>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_check = df_focalbirds.groupby(['PenID','RPenID'])['SireID','Rtreatment'].agg(lambda x: list(x)).reset_index()


,PenID,RPenID,SireID,Rtreatment,nbr_hens
0,pen10,Rpen11,"[67, 82, 34, 91, 67]","[Queue, Queue, Queue, Queue, Queue]",5
1,pen10,Rpen12,"[30, 19, 46, 6, 77, 47]","[Queue, Queue, Queue, Queue, Queue, Queue]",6
2,pen10,Rpen13,"[63, 78, 58, 6, 100]","[NoQueue, NoQueue, NoQueue, NoQueue, NoQueue]",5
3,pen10,Rpen14,"[87, 51, 97, 99, 7]","[NoQueue, NoQueue, NoQueue, NoQueue, NoQueue]",5
4,pen11,Rpen11,"[35, 75, 98, 53, 84]","[Queue, Queue, Queue, Queue, Queue]",5
5,pen11,Rpen12,"[8, 90, 24, 78, 8]","[Queue, Queue, Queue, Queue, Queue]",5
6,pen11,Rpen13,"[74, 44, 52, 39, 53, 19]","[NoQueue, NoQueue, NoQueue, NoQueue, NoQueue, ...",6
7,pen11,Rpen14,"[34, 9, 16, 63, 17]","[NoQueue, NoQueue, NoQueue, NoQueue, NoQueue]",5
8,pen12,Rpen11,"[32, 64, 54, 42, 89]","[Queue, Queue, Queue, Queue, Queue]",5
9,pen12,Rpen12,"[78, 26, 52, 34, 34]","[Queue, Queue, Queue, Queue, Queue]",5


In [124]:
#verify: no daughter from sire with almost no daughters
df_focalbirds[df_focalbirds['SireID'].isin([96,81,45,41,3114,18,66,88])]

,SireID,RPenID,PenID,legringID,Rtreatment,LPLegring


In [125]:
#verify: not more than avilable daughter from sire with almost no daughters (they have 2,2,3 duaghters reps)
df_focalbirds[df_focalbirds['SireID'].isin([5,11,80])]

,SireID,RPenID,PenID,legringID,Rtreatment,LPLegring
57,80,Rpen13,pen5,16,NoQueue,black


In [131]:
#verify: not more than avilable daughter from all sire we know all have more than 10
df_check = df_focalbirds['SireID'].value_counts().reset_index()
df_check.sort_values(['SireID']).tail(15)
#-->Sire 34 was fine I checked!

,index,SireID
8,78,4
10,21,4
9,19,4
12,36,4
13,39,4
14,44,4
15,1,4
11,26,4
6,13,5
5,7,5
